# Step 1: Data Discovery & Cleaning (Hypothesis 2)

**Goal:** Verify we can identify the "Leader" for every event and filter out sporadic leaders.

**Tasks:**
1. Inspect `events.json` and `event_user.json` to confirm leader ID fields.
2. Check for "Guest Leaders" (leaders with < 5 events).

In [ ]:
import pandas as pd
import numpy as np
import sys
import os

# Add src to path to import data_loader
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from data_loader import FitFamDataLoader

# Initialize Loader
loader = FitFamDataLoader(data_dir=os.path.abspath(os.path.join('..', 'fitfam-json')))

print("Loading Data...")
events = loader.load_events()
event_user = loader.load_event_user()
users = loader.load_users()
print("Data Loaded.")

## 1. Identify Leader ID in `events` table

In [ ]:
print("Events Columns:", events.columns.tolist())
events.head(3)

## 2. Identify Leader in `event_user` table (Attendance)

In [ ]:
print("Event_User Columns:", event_user.columns.tolist())
event_user.head(3)

In [ ]:
# Check for 'is_leader' column
if 'is_leader' in event_user.columns:
    print("is_leader unique values:", event_user['is_leader'].unique())
    print(event_user['is_leader'].value_counts())
else:
    print("'is_leader' column NOT found")

## 3. Extract Leaders & Filter Guest Leaders

In [ ]:
if 'is_leader' in event_user.columns:
    leaders_from_attendance = event_user[event_user['is_leader'] == 1]
    print(f"Found {len(leaders_from_attendance)} leader records in event_user.")
    
    if not leaders_from_attendance.empty:
        leader_counts = leaders_from_attendance.groupby('user_id').size().reset_index(name='event_count')
        print(f"Total unique leaders found: {len(leader_counts)}")
        
        # Filter Active Leaders (>= 5 events)
        active_leaders = leader_counts[leader_counts['event_count'] >= 5]
        print(f"Active Leaders (>= 5 events): {len(active_leaders)}")
        
        print("\nLeader Event Count Distribution:")
        print(leader_counts['event_count'].describe())
else:
    print("Cannot identify leaders from event_user.")

# Leader Quality Metrics

Create a `df_leaders` table with quality metrics for each leader.

**Metrics:**
1. **Consistency:** Std Dev of days between led events.
2. **Tenure:** Days between first and last led event.
3. **Frequency:** Events per month.

In [ ]:
# 1. Merge Leaders with Event Dates
# We need the 'start_time' of every event they led
leaders = event_user[event_user['is_leader'] == 1].copy()
leaders_with_dates = leaders.merge(events[['id', 'start_time']], left_on='event_id', right_on='id', how='left')

# Ensure datetime
leaders_with_dates['start_time'] = pd.to_datetime(leaders_with_dates['start_time'])

# Sort
leaders_with_dates = leaders_with_dates.sort_values(['user_id', 'start_time'])

print("Merged leader attendance with event dates.")
leaders_with_dates.head()

In [ ]:
# 2. Calculate Metrics Loop
leader_stats = []

print("Calculating metrics for each leader...")

for user_id, group in leaders_with_dates.groupby('user_id'):
    dates = group['start_time'].sort_values()
    
    # Basic Stats
    total_events = len(group)
    first_led = dates.iloc[0]
    last_led = dates.iloc[-1]
    
    # Tenure (Days)
    tenure_days = (last_led - first_led).days
    # Avoid zero division for single-day leaders
    if tenure_days == 0: 
        tenure_days = 1 
    
    # Frequency (Events per Month)
    # 30.44 days per month on average
    events_per_month = total_events / (tenure_days / 30.44)
    
    # Consistency (Std Dev of gaps)
    if total_events > 1:
        # Calculate difference in days between consecutive events
        gaps = dates.diff().dt.total_seconds() / (24 * 3600)
        gaps = gaps.dropna()
        consistency_std = gaps.std()
        avg_gap = gaps.mean()
    else:
        consistency_std = np.nan # Undefined for single events
        avg_gap = np.nan
        
    leader_stats.append({
        'leader_user_id': user_id,
        'total_events': total_events,
        'first_led_date': first_led,
        'last_led_date': last_led,
        'tenure_days': tenure_days,
        'events_per_month': events_per_month,
        'consistency_std': consistency_std,
        'avg_gap_days': avg_gap
    })

df_leaders = pd.DataFrame(leader_stats)
df_leaders.set_index('leader_user_id', inplace=True)

print(f"Computed stats for {len(df_leaders)} leaders.")

In [ ]:
# 3. Inspect the Results
print("Top 5 Consistent Leaders (Lowest Std Dev, min 10 events):")
print(df_leaders[df_leaders['total_events'] >= 10].sort_values('consistency_std').head(5)[['total_events', 'consistency_std', 'events_per_month']])

print("\nTop 5 Most Frequent Leaders:")
print(df_leaders.sort_values('events_per_month', ascending=False).head(5)[['total_events', 'consistency_std', 'events_per_month']])

In [ ]:
# 4. Save/Export for next steps
# We will use this df_leaders in Step 3
df_leaders.to_csv('leaders_quality_metrics.csv')
print("Saved leader metrics to leaders_quality_metrics.csv")